In [21]:
!pip install -q pytelegrambotapi pydub

In [22]:
import os
import telebot
import numpy as np
from pydub import AudioSegment, silence, utils
from tensorflow.keras.models import load_model

In [ ]:
TOKEN = '7288651187:AAGntB3njMyb7FUeC7p4uPo8KBPw_pdo96E'
bot = telebot.TeleBot(TOKEN)
model = load_model('voice_model.h5')

In [24]:
classes = ['A_Ramezani', 'AmirHosein_Molaei', 'Davood_Fazeli', 'Javad_Nematollahi', 'Khadijeh_Valipour', 'Kiana_Jahanshid', 'Maryam_Saeedi', 'Matin_Ghorbani', 'Moein_Moatali', 'Mohamad_parvari', 'Mohammad_nematizadeh', 'Mona_nematizadeh', 'Nahid_Ebrahimian', 'Nima', 'Sajedeh_Gharabadiyan', 'Shima_Bazzazan']

In [25]:
def preprocess_chunk(chunk):
    chunk = chunk.set_sample_width(2)
    chunk = chunk.set_frame_rate(48000)
    chunk = chunk.set_channels(1)

    samples = np.array(chunk.get_array_of_samples()).astype(np.float32) / 32768.0
    if len(samples) < 48000:
        samples = np.pad(samples, (0, 48000 - len(samples)))
    else:
        samples = samples[:48000]

    return np.expand_dims(samples, axis=0)

In [26]:
@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    ogg_path = f"{message.from_user.id}.ogg"
    wav_path = f"{message.from_user.id}.wav"

    with open(ogg_path, 'wb') as f:
        f.write(downloaded_file)

    audio = AudioSegment.from_ogg(ogg_path)

    chunks = silence.split_on_silence(
        audio,
        min_silence_len=1000,
        silence_thresh=-40
    )

    if not chunks:
        bot.reply_to(message, "I couldn't process your voice, Please try again!")
        return
    audio_processed = sum(chunks)

    chunks = utils.make_chunks(audio_processed, 1000)

    predictions = []
    for chunk in chunks:
        if len(chunk) >= 1000:
            input_data = preprocess_chunk(chunk)
            pred = model.predict(input_data)
            pred_class = np.argmax(pred)
            predictions.append(pred_class)

    if not predictions:
        bot.reply_to(message, "Error")
        return

    from collections import Counter
    most_common_class = Counter(predictions).most_common(1)[0][0]

    bot.reply_to(message, f"Model predicted: {classes[most_common_class]}")


In [ ]:
bot.polling()